In [90]:
#Ahmed Adel Ahmed Oraby (20210550)

#[1] import libraries
import re
from sympy.logic.boolalg import to_cnf

#[2] Eliminate implication.
def eliminate_implication(sentence):
    """
    Function to eliminate implications in a logical sentence.
    """
    # Remove the outer brackets if they exist
    if sentence.startswith('(') and sentence.endswith(')'):
        sentence = sentence[1:-1]

    # Find the index of '=>'
    imp_index = sentence.find('=>')

    # Extract the antecedent and consequent parts
    antecedent = sentence[:imp_index].strip()
    consequent = sentence[imp_index + 2:].strip()

    # Replace implication with equivalent form ~p | q
    return f"(¬{antecedent} ∨ {consequent})"



#[3] Demorgans_Law
def move_negation_inward(sentence):
    """
    Function to move negation inward using De Morgan's Law.
    """
    # Apply De Morgan's Law: ~(p ∨ q) = (~p ∧ ~q) and ~(p ∧ q) = (~p ∨ ~q)
    while "¬(" in sentence:
        negation_index = sentence.find("¬(")
        closing_index = negation_index + 2
        opened = 1
        while opened != 0:
            if sentence[closing_index] == '(':
                opened += 1
            elif sentence[closing_index] == ')':
                opened -= 1
            closing_index += 1

        inner_expression = sentence[negation_index + 2:closing_index - 1]
        negated_inner_expression = ""
        for char in inner_expression:
            if char == '∨':
                negated_inner_expression += '∧'
            elif char == '∧':
                negated_inner_expression += '∨'
            elif char.isalpha():
                negated_inner_expression += '¬' + char

        sentence = sentence[:negation_index] + negated_inner_expression + sentence[closing_index:]

    return sentence



#[3] Remove_double-not.
def remove_double_not(sentence):
    return sentence.replace("¬¬", "")


#[5] Standardize variable scope
def Standardize_variable_scope(s):
    vars = ['none','w', 'x', 'y', 'z']
    new_string = ""
    i = 0
    for ch in s:
        if ch == '∃' or ch == '∀':
            i += 1
        if ch in vars:
            ch = vars[i]
        new_string += ch
    return new_string

#[6] prenex form
def prenex_form(s):
    quantifiers = ""
    new_string = ""
    skip_next = False
    for i, ch in enumerate(s):
        if skip_next:
            skip_next = False
            continue
        if ch in ['∃', '∀']:
            quantifiers += ch + s[i + 1]
            skip_next = True
        else:
            new_string += ch
    return quantifiers + new_string

def Skolemization(s):
    result_string = ""
    prefix_vars = []  # To store variables introduced by universal quantifiers (∀)
    existential_vars = []  # To store variables that need to be replaced because they're introduced by existential quantifiers (∃)

    # First pass: Identify and process quantifiers and variables
    i = 0
    while i < len(s):
        if s[i] == '∀':
            prefix_vars.append(s[i+1])  # Add variable after ∀ to the prefix list
            result_string += s[i] + s[i+1]  # Keep ∀ and its variable in the result string
            i += 2  # Skip next symbol (variable)
        elif s[i] == '∃':
            existential_vars.append(s[i+1])  # Add variable after ∃ to be replaced later
            i += 2  # Skip ∃ and its variable, not adding them to result_string
        else:
            result_string += s[i]
            i += 1

    # Second pass: Replace existential variables with Skolem functions
    final_result = ""
    for ch in result_string:
        if ch in existential_vars:
            if prefix_vars:
                skolem_function = f'F({",".join(prefix_vars)})'
            else:
                skolem_function = 'F()'
            final_result += skolem_function
        else:
            final_result += ch

    return final_result

#[8] Eliminate universal quantifiers
def Eliminate_universal_quantifiers(s):
    pattern = r'∀[a-zA-Z]'
    new_expression = re.sub(pattern, '', s)
    return new_expression


#[9] Convert to conjunctive normal form
def Convert_to_conjunctive_normal_form(s):
    new_string = eliminate_implication(s)
    new_string = move_negation_inward(new_string)
    new_string = remove_double_not(new_string)
    return new_string

#[10]Turn conjunctions into clauses in a set, and rename variables so that no clause shares the same variable name
def Turn_conjunctions(s):
    return s.split("^")

#[11] Rename variables in clauses so that each clause has a unique variable name
def Rename_variables(lst):
    vars = ["w", "x", "y", "z"]
    i =0
    for clause in range(len(lst)):
        # lst[clause].replace(var in vars, vars[i])
        cls =lst[clause]
        for ch in cls:
            if ch in vars:
                cls = cls.replace(ch,vars[i])
        lst[clause] = cls
        i+=1
    return lst


#[12] Example

logical_expression = "∀x(P(x)→∃xQ(x))"
print(logical_expression)
step1_result = eliminate_implication(logical_expression)
print("After Eliminate_implication : ", step1_result)

step2_result = move_negation_inward(step1_result)
print("After Demorgan : ", step2_result)

step3_result = remove_double_not(step2_result)
print("After Remove_double_not : ", step3_result)

step4_result = Standardize_variable_scope(logical_expression)
print("After Standardize_variable_scope : ", step4_result)

step5 = prenex_form(step4_result)
print("before Skolemization : ", step5)

step6 = Skolemization(step5)
print("After Skolemization : ",step6)

step7 = Eliminate_universal_quantifiers(step6)
print("After Eliminate_universal_quantifiers : ", step7)

step8 = Convert_to_conjunctive_normal_form(step7)
print("After Convert_to_conjunctive_normal_form : ",step8)

step9 = Turn_conjunctions(step8)
print("After Turn_conjunctions : ",step9)

step10 = Rename_variables(step9)
print("After renaming : ",step10)

∀x(P(x)→∃xQ(x))
After Eliminate_implication :  (¬∀x(P(x)→∃xQ(x) ∨ x(P(x)→∃xQ(x)))
After Demorgan :  (¬∀x(P(x)→∃xQ(x) ∨ x(P(x)→∃xQ(x)))
After Remove_double_not :  (¬∀x(P(x)→∃xQ(x) ∨ x(P(x)→∃xQ(x)))
After Standardize_variable_scope :  ∀w(P(w)→∃xQ(x))
before Skolemization :  ∀w∃x(P(w)→Q(x))
After Skolemization :  ∀w(P(w)→Q(F(w)))
After Eliminate_universal_quantifiers :  (P(w)→Q(F(w)))
After Convert_to_conjunctive_normal_form :  (¬P(w)→Q(F(w) ∨ (w)→Q(F(w)))
After Turn_conjunctions :  ['(¬P(w)→Q(F(w) ∨ (w)→Q(F(w)))']
After renaming :  ['(¬P(w)→Q(F(w) ∨ (w)→Q(F(w)))']
